In [9]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as stats

In [13]:
max_gens = 200
population_files = [f for f in os.listdir('../../') if f.endswith('.dat') and f.startswith('population')]
population_files = sorted(population_files, key=lambda x: int(x.split('_')[3].split('.')[0]))[:max_gens]
selection_files = [f for f in os.listdir('../../') if f.endswith('.dat') and f.startswith('selection')]
selection_files = sorted(selection_files, key=lambda x: int(x.split('_')[3].split('.')[0]))[:max_gens]
ams_files = [f for f in os.listdir('../../') if f.endswith('.dat') and f.startswith('ams')]
ams_files = sorted(ams_files, key=lambda x: int(x.split('_')[3].split('.')[0]))[:max_gens]
cov_files = [f for f in os.listdir('../../') if f.endswith('.dat') and f.startswith('cov')]
cov_files = sorted(cov_files, key=lambda x: int(x.split('_')[3].split('.')[0]))[:max_gens]


In [14]:
import math

def norm_pdf_multivariate(x, mu, sigma):
    size = x.shape[-1]
    if size == len(mu) and (size, size) == sigma.shape:
        det = np.linalg.det(sigma)
        if det == 0:
            raise NameError("The covariance matrix can't be singular")

        norm_const = 1.0/ ( math.pow((2*math.pi),float(size)/2) * math.pow(det,1.0/2) )
        x_mu = np.matrix(x - mu)
        inv = sigma.I        
        result = math.pow(math.e, -0.5 * (x_mu * inv * x_mu.T))
        return norm_const * result
    else:
        raise NameError("The dimensions of the input don't match")
    
def est_mult_gaus(X,mu,sigma):
    m = len(mu)
    sigma2 = np.diag(sigma)
    X = X-mu.T
    p = 1/((2*np.pi)**(m/2)*np.linalg.det(sigma2)**(0.5))*np.exp(-0.5*np.sum(X.dot(np.linalg.pinv(sigma2))*X,axis=1))

    return p

import scipy.sparse as sp
import scipy.sparse.linalg as spln
def lognormpdf(x,mu,S):
    """ Calculate gaussian probability density of x, when x ~ N(mu,sigma) """
    nx = len(S)
    norm_coeff = nx*math.log(2*math.pi)+np.linalg.slogdet(S)[1]

    err = x-mu
    if (sp.issparse(S)):
        numerator = spln.spsolve(S, err).T.dot(err)
    else:
        numerator = np.linalg.solve(S, err).T.dot(err)

    return -0.5*(norm_coeff+numerator)

In [ ]:
import multiprocessing as mp
dir = 'figures_inc_small_overlap'
if not os.path.exists(dir):
    os.makedirs(dir)
else:
    raise "WARNING directory "+dir+" already exists"

def plot_cov_ellipse(files):
    [pop_file, sel_file, ams_file, cov_file] = files
    n_blocks = 10
    block_size = 2
    fig, axs = plt.subplots(1, n_blocks, figsize=(30,5))#, sharex=True, sharey=True)
    gen = pop_file.split('_')[3].split('.')[0]
    fig.suptitle('Generation ' + gen)
    pop = pd.read_csv('../../' + pop_file, sep='\s+', header=None)
    sel = pd.read_csv('../../' + sel_file, sep='\s+', header=None)
    ams = pd.read_csv('../../' + ams_file, sep='\s+', header=None)
    cov = pd.read_csv('../../' + cov_file, sep='\s+', header=None)
    for i in range(n_blocks):
        idxx = i * block_size
        pop.plot(x=idxx, y=idxx+1, kind='scatter', color="black", ax = axs[i])
        sel.plot(x=idxx, y=idxx+1, kind='scatter', color="green", ax = axs[i])
        
        # Prevents errors when the covariance matrix is not positive definite
        cov_matrix = cov.loc[[idxx, idxx+1], [idxx, idxx+1]].to_numpy()
        # print(cov_matrix)
        # print(np.linalg.eigvals(cov_matrix))
        if np.all(np.linalg.eigvals(cov_matrix) > 0):
            rv = stats.multivariate_normal([sel[idxx].mean(), sel[idxx+1].mean()], cov_matrix, True)
        else:
            # print(cov_matrix + np.eye(2) * (np.min(np.linalg.eigvals(cov_matrix)) + 1e-12))
            # print(np.linalg.eigvals(cov_matrix + np.eye(2) * ((np.min(np.linalg.eigvals(cov_matrix)) * -1) + 1e-12)))
            rv = stats.multivariate_normal([sel[idxx].mean(), sel[idxx+1].mean()], cov_matrix + np.eye(2) * ((np.min(np.linalg.eigvals(cov_matrix)) * -1) + 1e-12), True)
        width = 0.001
        
        if (sel[idxx].max() - sel[idxx].min()) > 10 or (sel[idxx+1].max() - sel[idxx+1].min()) > 10 :
            width = 0.1
        x, y = np.mgrid[sel[idxx].min()-1:sel[idxx].max()+1:.1, sel[idxx+1].min()-1:sel[idxx+1].max()+1:.1]
        data = np.dstack((x, y))
        z = rv.pdf(data)
        axs[i].contour(x, y, z, cmap='coolwarm')
        axs[i].arrow(sel[idxx].mean(), sel[idxx+1].mean(), ams[idxx].to_numpy()[0], ams[idxx+1].to_numpy()[0], color='red', width=width )
        axs[i].grid()
    plt.savefig(dir+'/gen_' + gen + '.png')
    # plt.show()
    plt.close()

with mp.Pool(24) as pool:
    pool.map(plot_cov_ellipse, zip(population_files, selection_files, ams_files, cov_files), chunksize=1)
    